<a href="https://colab.research.google.com/github/fspanda/Electra_Further_Training/blob/main/Electra_Further_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.15
!pip install transformers==2.8.0
!git clone https://github.com/google-research/electra.git
!pip install urllib3==1.25.10

     |█████████████████▌              | 225.4MB 1.2MB/s eta 0:02:34

In [ ]:
import os
import json
from transformers import AutoTokenizer
import sys
import nltk
import random
import logging
import tensorflow as tf


from glob import glob
from google.colab import auth, drive

auth.authenticate_user()

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [ ]:
!mkdir data
!split -a 4 -l 256000 -d '/content/drive/My Drive/Study/Medical-Corpus.txt' ./data/data_
!ls data

data_0000  data_0002  data_0004  data_0006  data_0008  data_0010  data_0012
data_0001  data_0003  data_0005  data_0007  data_0009  data_0011


In [ ]:
os.getcwd()

'/content'

In [ ]:
VOC_FNAME = "vocab.txt"
PRETRAINING_DIR = "pretraining_data"

XARGS_CMD = ("ls ./data/ | "
             "xargs -n 1 -P 4 -I{} "
             "python3 /content/electra/build_pretraining_dataset.py "
             "--corpus-dir=data " 
             "--vocab-file={} " 
             "--output-dir={} "
             "--max-seq-length=128"
             "--num-processes=4"
             "--no-lower-case ")
XARGS_CMD=XARGS_CMD.format({},VOC_FNAME,PRETRAINING_DIR)



tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD

usage: build_pretraining_dataset.py [-h] --corpus-dir CORPUS_DIR --vocab-file
                                    VOCAB_FILE --output-dir OUTPUT_DIR
                                    [--max-seq-length MAX_SEQ_LENGTH]
                                    [--num-processes NUM_PROCESSES]
                                    [--blanks-separate-docs BLANKS_SEPARATE_DOCS]
                                    [--do-lower-case] [--no-lower-case]
build_pretraining_dataset.py: error: argument --num-processes: invalid int value: '4--no-lower-case'
usage: build_pretraining_dataset.py [-h] --corpus-dir CORPUS_DIR --vocab-file
                                    VOCAB_FILE --output-dir OUTPUT_DIR
                                    [--max-seq-length MAX_SEQ_LENGTH]
                                    [--num-processes NUM_PROCESSES]
                                    [--blanks-separate-docs BLANKS_SEPARATE_DOCS]
                                    [--do-lower-case] [--no-lower-case]
build_pretraining

In [ ]:
!gsutil -m mv -r pretraining_data gs://electra_medical_v2/Electra/pretrain_tfrecords

In [ ]:
BUCKET_NAME = "electra-medica" #@param {type:"string"}
MODEL_DIR = "electra-medical" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)

In [ ]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

TPU address is =>  grpc://10.38.4.250:8470


In [ ]:
if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

CommandException: "cp" command does not support provider-only URLs.


In [ ]:
base_config = {
  "tpu_name": "electra-medica",
  "vocab_size": 33000,
  "num_train_steps": 700000,
  "save_checkpoints_steps": 25000,
  "train_batch_size": 256,
  "learning_rate": 2e-4
}

with open("{}/base_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(base_config, fo, indent=2)

In [ ]:
 # Upload credentials to TPU.
with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)


In [ ]:
!cd electra

In [ ]:
!python3 /content/electra/run_pretraining.py --data-dir gs://{BUCKET_NAME} \ --model-name: {"electra-medical"}\ --haparams: {"adc.json"}

usage: run_pretraining.py [-h] --data-dir DATA_DIR --model-name MODEL_NAME
                          [--hparams HPARAMS]
run_pretraining.py: error: the following arguments are required: --model-name


In [ ]:
XARGS_CMD = ("python3 /content/electra/run_pretraining.py"
             "--data-dir={} " 
             "--model-name={} " 
             "--haparams={} "
             )
XARGS_CMD=XARGS_CMD.format(BUCKET_NAME,"electra-medical","adc.json")
!$XARGS_CMD

python3: can't open file '/content/electra/run_pretraining.py--data-dir=electra-medica': [Errno 2] No such file or directory


In [ ]:
XARGS_

In [ ]:
import pandas as pd
train_data=pd.read_table("/content/drive/My Drive/Study/Medical-Corpus.txt")
vocab_data=pd.read_table("/content/drive/My Drive/Study/Medical_vocab_1.txt",header=None)
train_data.to_csv("/content/data/train_data.txt",index=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
tokenizer.save_pretrained(DATA_DIR)


('./data/vocab.txt',
 './data/special_tokens_map.json',
 './data/added_tokens.json')

In [ ]:
vocab_data.to_csv("/content/data/vocab.txt",index=False,header=None)

In [ ]:
def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count
RAW_DATA_FPATH = "/content/data/train_data.txt" #@param {type: "string"}
PRC_DATA_FPATH = "/content/data/proc_dataset.txt" #@param {type: "string"}

regex_tokenizer = nltk.RegexpTokenizer("\w+")
def normalize_text(text):
  # lowercase text
  #text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punktuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

# apply normalization to the dataset
# this will take a minute or two


total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

3124556/3124556 [==============================] - 36s 11us/step


In [ ]:
!mkdir ./shards_electra
!split -a 4 -l 256000 -d $PRC_DATA_FPATH ./shards_electra/shard_
!ls ./shards_electra/

shard_0000  shard_0003	shard_0006  shard_0009	shard_0012
shard_0001  shard_0004	shard_0007  shard_0010
shard_0002  shard_0005	shard_0008  shard_0011


In [ ]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = False #@param {type:"boolean"}
PROCESSES = 4 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}


In [ ]:
!python3 electra/build_pretraining_dataset.py \
  --corpus-dir $DATA_DIR \
  --vocab-file $DATA_DIR/vocab.txt \
  --output-dir $DATA_DIR/pretrain_tfrecords \
  --max-seq-length 128 \
  --blanks-separate-docs False \
  --no-lower-case \
  --num-processes 5

Job 0: Creating example writer
Job 1: Creating example writer
Job 2: Creating example writer
Job 3: Creating example writer
Job 4: Creating example writer
Job 1: Writing tf examples
Job 0: Writing tf examples
Job 2: Writing tf examples
Job 2: Done!
Job 3: Writing tf examples
Job 4: Writing tf examples
Job 3: Done!
Job 0: processed 1/2 files (50.0%), ELAPSED: 0s, ETA: 0s, 474 examples written
Job 0: Done!
Job 1: Done!
Job 4: Done!


In [ ]:
len(voc_data)

In [ ]:
hparams = {
    "do_train": "true",
    "do_eval": "false",
    "model_size": "small",
    "do_lower_case": "false",
    "vocab_size": 33000,
    "num_train_steps": 100,
    "save_checkpoints_steps": 100,
    "train_batch_size": 32,
}
           
with open("hparams.json", "w") as f:
    json.dump(hparams, f)

In [ ]:
BUCKET_NAME = "electra-medica" #@param {type:"string"}
MODEL_DIR = "electra-medical" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)

In [ ]:
with open("{}/config.json".format(MODEL_DIR), "w") as fo:
  json.dump(config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [ ]:
if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME


CommandException: No URLs matched: pretraining_data
Copying file://bert_model/vocab.txt [Content-Type=text/plain]...
Copying file://bert_model/bert_config.json [Content-Type=application/json]...
/ [2/2 files][317.3 KiB/317.3 KiB] 100% Done                                    
Operation completed over 2 objects/317.3 KiB.                                    
CommandException: 1 file/object could not be transferred.


In [ ]:
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = "vocab.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 10000000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2020-10-24 00:53:30,900 :  Using checkpoint: gs://bert_further_training/bert_model/model.ckpt-1000000
2020-10-24 00:53:30,901 :  Using 13 data shards


In [ ]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)


2020-10-24 00:52:08,767 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f08e6356268>) includes params argument, but params are not passed to Estimator.
2020-10-24 00:52:08,768 :  Using config: {'_model_dir': 'gs://bert_further_training/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.73.140.146:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07f3c7e630>, '_task_type': 'worker', '_task_id'

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2020-10-24 00:53:34,925 :  Querying Tensorflow master (grpc://10.73.140.146:8470) for TPU system metadata.
2020-10-24 00:53:34,934 :  Found TPU system:
2020-10-24 00:53:34,935 :  *** Num TPU Cores: 8
2020-10-24 00:53:34,935 :  *** Num TPU Workers: 1
2020-10-24 00:53:34,935 :  *** Num TPU Cores Per Worker: 8
2020-10-24 00:53:34,936 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6546476083521642609)
2020-10-24 00:53:34,937 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12674493085490563589)
2020-10-24 00:53:34,938 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8131706013016610670)
2020-10-24 00:53:34,939 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2648248771191916361)
2020-10-24 00:53:34,939 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:

2020-10-24 00:53:35,638 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35ace10>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35ace10>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:35,774 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35acc18>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35acc18>>: AssertionError: Bad argument number for

2020-10-24 00:53:35,882 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f352b5f8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f352b5f8>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:36,015 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f32e5710>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f32e5710>>: AssertionError: Bad argument number for

2020-10-24 00:53:36,114 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f3244cf8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f3244cf8>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:36,212 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f3244cf8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f3244cf8>>: AssertionError: Bad argument number for

2020-10-24 00:53:36,350 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35ac518>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f35ac518>>: AssertionError: Bad argument number for Name: 3, expecting 4


...

2020-10-24 00:53:43,685 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f139c518>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f139c518>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:43,785 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f139c518>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f139c518>>: AssertionError: Bad argument number for

2020-10-24 00:53:43,919 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f13f49b0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f13f49b0>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:44,057 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f2635b00>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f2635b00>>: AssertionError: Bad argument number for

2020-10-24 00:53:44,169 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f2c076d8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f2c076d8>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-10-24 00:53:44,330 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f36f82e8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f36f82e8>>: AssertionError: Bad argument number for

2020-10-24 00:53:44,443 :  Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f0fa4e80>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f07f0fa4e80>>: AssertionError: Bad argument number for Name: 3, expecting 4


2020-10-24 00:53:44,687 :  **** Trainable Variables ****
2020-10-24 00:53:44,688 :    name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
2020-10-24 00:53:44,688 :    name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
2020-10-24 00:53:44,689 :    name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
2020-10-24 00:53:44,689 :    name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
2020-10-24 00:53:44,690 :    name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
2020-10-24 00:53:44,691 :    name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
2020-10-24 00:53:44,691 :    name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
2020-10-24 00:53:44,692 :    name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
2020-10-24 00:53:44,692 :    name =